In [4]:
import chromadb
from openapiutils import download_openapi_spec, get_function_names_from_file, create_function_docs_and_import_statements, clean_url, generate_hash

chroma_client = chromadb.PersistentClient(path="agentprotocol/vectordb")


In [ ]:
doc2 = f'''
Function:
def give_joke(category : str):
"""
Joke categories. Supports: Any, Misc, Programming, Dark, Pun, Spooky, Christmas.
"""
'''

In [5]:
openapi_url = "https://raw.githubusercontent.com/open-meteo/open-meteo/main/openapi.yml"
service_url = "https://api.open-meteo.com"
output_dir = generate_hash(clean_url(openapi_url))
download_openapi_spec(openapi_url, service_url)

Downloaded and modified OpenAPI spec saved to: openapi.json


In [6]:
import subprocess

# Construct the command using string formatting
command = f"openapi-python-generator openapi.json {output_dir}/"  # !openapi-python-generator openapi.json ./api_specification_main/
subprocess.run(command.split(), check=True)

CompletedProcess(args=['openapi-python-generator', 'openapi.json', 'a41cc9d3b6585b73a594eb1cb0cab5fcd3f67dad09b45c4572583da07b3cc751/'], returncode=0)

In [7]:
import os
os.remove("openapi.yml")
os.remove("openapi.json")

In [8]:
function_docs, import_statements = create_function_docs_and_import_statements(output_dir)

Creating function docs


In [9]:
doc_ids = []
doc_metadatas = []
for function_doc in function_docs:
    doc_ids.append(str(generate_hash(function_doc)))
    doc_metadatas.append({"uri": "", "type": "native"})
    print(str(generate_hash(function_doc)))

ca264c26145649218b871d756106fd3423dca0d72666986a06798e20a5d2d285


In [10]:
collection = chroma_client.get_or_create_collection(name="tools")
collection.add(
    documents=function_docs,
    metadatas=doc_metadatas,
    ids=doc_ids
)


In [11]:
user_query = "Hey how is the current weather and windspeed in New York?"

In [12]:
prompt = f'''
    Function:
    no_op():
    """
    Default OOD
    """
    '''

In [13]:
results = collection.query(
    query_texts=[user_query], 
    n_results=2, # how many results to return
    # include=["embeddings", "metadatas", "documents", "distances"]
    include=["documents"]
)

for items in results["documents"]:
    for item in items:
        # print(str(item))
        prompt += str(item) + "\n"

prompt = prompt + f'''User Query: {user_query}<human_end>'''
print(prompt)


Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1



    Function:
    no_op():
    """
    Default OOD
    """
    
    Function:
    get_v1forecast(latitude: float, longitude: float, hourly: Optional[List[str]] = None, daily: Optional[List[str]] = None, current_weather: Optional[bool] = None, temperature_unit: Optional[str] = None, wind_speed_unit: Optional[str] = None, timeformat: Optional[str] = None, timezone: Optional[str] = None, past_days: Optional[int] = None, api_config_override: Optional[a41cc9d3b6585b73a594eb1cb0cab5fcd3f67dad09b45c4572583da07b3cc751.api_config.APIConfig] = None) -> Dict[str, Any]
    """
    Requires the latitude and longitude.
    Set current_weather to True to get the weather.
    Set hourly or daily based on preference.
    """
User Query: Hey how is the current weather and windspeed in New York?<human_end>


In [14]:
from utils import query_raven
call = query_raven(prompt)
print (call)

c:\Projects\Old\AgenticProtocol\function_calling_llm\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


get_v1forecast(latitude=40.7128, longitude=-74.0060, current_weather=True)


In [16]:
for import_statement in import_statements:
    exec(import_statement)
eval(call)

{'latitude': 40.710335,
 'longitude': -73.99309,
 'generationtime_ms': 0.08296966552734375,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 32.0,
 'current_weather_units': {'time': 'iso8601',
  'interval': 'seconds',
  'temperature': '°C',
  'windspeed': 'km/h',
  'winddirection': '°',
  'is_day': '',
  'weathercode': 'wmo code'},
 'current_weather': {'time': '2024-07-05T15:45',
  'interval': 900,
  'temperature': 28.9,
  'windspeed': 8.8,
  'winddirection': 192,
  'is_day': 1,
  'weathercode': 0}}

In [ ]:
count = len(chroma_client.list_collections())
print(count)
if count > 0:
    print(chroma_client.list_collections())

In [17]:
chroma_client.delete_collection("tools")

In [18]:
directory_path = 'a41cc9d3b6585b73a594eb1cb0cab5fcd3f67dad09b45c4572583da07b3cc751'
import shutil
if os.path.exists(directory_path):
    shutil.rmtree(directory_path)